In [30]:
# import keras
import fastai
from fastai.vision import *
# from fastai.metrics import fbeta, Precision, Recall, accuracy_thresh

In [33]:
fastai.__version__

'1.0.34'

In [11]:
import pandas as pd
import numpy as np

import os
import shutil
from pathlib import Path
import tarfile
import re

import matplotlib.pyplot as plt
from PIL import Image

In [62]:
path_data = Path('../data')

df_labs = pd.read_csv(path_data/'ComParE2018_AtypicalAffect.txt', sep='\t')

df_labs['file_name'] = df_labs['file_name'].str.replace('wav', 'png')

df_labs['subset'] = df_labs['file_name'].str.extract(r'([a-z]+)')
df_labs['subset'][df_labs['subset'] == 'devel'] = 'Valid'
df_labs['subset'][df_labs['subset'] == 'train'] = 'Train' 
df_labs['subset'][df_labs['subset'] == 'test'] = 'Test' 
# df_labs['subset'].value_counts()
df_labs.head()

,file_name,emotion,subset
0,train_0001.png,neutral,Train
1,train_0002.png,neutral,Train
2,train_0003.png,neutral,Train
3,train_0004.png,neutral,Train
4,train_0005.png,neutral,Train


In [78]:
tfms = get_transforms(do_flip=False, flip_vert=False, max_rotate=0.02)

basic_block = (fastai.vision.ImageItemList
               .from_df(path=path_data/'spectogram', df=df_labs.query('subset!="Test"'), label_col='emotion')
               .split_by_idx(valid_idx=df_labs.query('subset=="Valid"').index)
               .label_from_df(cols=1)
              )

db_speech = (basic_block
            .transform(tfms, 
                       size=64) 
            .databunch(bs=32))

db_speech.normalize(imagenet_stats)

# db_speech.export()



ImageDataBunch;

Train: LabelList
y: CategoryList (3342 items)
[Category neutral, Category neutral, Category neutral, Category neutral, Category neutral]...
Path: ../data/spectogram
x: ImageItemList (3342 items)
[Image (3, 217, 223), Image (3, 217, 223), Image (3, 217, 223), Image (3, 217, 223), Image (3, 217, 223)]...
Path: ../data/spectogram;

Valid: LabelList
y: CategoryList (3000 items)
[Category happy, Category happy, Category neutral, Category neutral, Category happy]...
Path: ../data/spectogram
x: ImageItemList (3000 items)
[Image (3, 217, 223), Image (3, 217, 223), Image (3, 217, 223), Image (3, 217, 223), Image (3, 217, 223)]...
Path: ../data/spectogram;

Test: None

In [79]:
db_speech.show_batch()

RuntimeError: Traceback (most recent call last):
  File "/home/gurudev/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/gurudev/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/gurudev/anaconda3/lib/python3.6/site-packages/fastai/data_block.py", line 486, in __getitem__
    x = x.apply_tfms(self.tfms, **self.tfmargs)
  File "/home/gurudev/anaconda3/lib/python3.6/site-packages/fastai/vision/image.py", line 113, in apply_tfms
    else: x = tfm(x)
  File "/home/gurudev/anaconda3/lib/python3.6/site-packages/fastai/vision/image.py", line 498, in __call__
    return self.tfm(x, *args, **{**self.resolved, **kwargs}) if self.do_run else x
  File "/home/gurudev/anaconda3/lib/python3.6/site-packages/fastai/vision/image.py", line 445, in __call__
    if a